# Deploy HLS Foundation Model into Vertex AI 

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
azure-storage-blob>=12.0.0
azure-ai-ml>=1.0.0
azure-identity>=1.0.0
mlflow>=2.0.0
torch>=1.9.0
torchvision>=0.10.0

In [ ]:
# Clone github with inference code for HLS foundation model
! git clone https://github.com/nasa-impact/hls-foundation-vertexai.git

In [ ]:
# Navigate inside hls-foundation-vertexai folder
import os
os.chdir('hls-foundation-vertexai')

In [ ]:
# Install required packages for Azure ML
! pip install -U azure-ai-ml azure-identity azure-storage-blob mlflow -q --user

# Install requirements from the cloned repo
! pip install -U -r requirements.txt -q --user

# For local FastAPI development and running
! pip install -U "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 -q --user

In [ ]:
# Check Azure CLI authentication status
! az account show

In [ ]:
# List available Azure subscriptions
! az account list --output table

In [ ]:
# Replace <subscription-id> with your subscription ID
! az account set --subscription "<subscription-id>"

## Use python to deploy HLS foundation model to vertexai endpoint 

In [ ]:
# Azure Configuration
import os
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    Model,
    Environment,
    CodeConfiguration,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings
)

# Replace with your Azure subscription and resource details
subscription_id = "<your-subscription-id>"
resource_group_name = "<your-resource-group>"
workspace_name = "<your-workspace-name>"

# Initialize Azure ML client
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
    workspace_name=workspace_name
)

print(f"Connected to workspace: {ml_client.workspace_name}")

In [ ]:
# Build and push Docker image to Azure Container Registry
# Replace <acr-name> with your Azure Container Registry name
acr_name = "<acr-name>"
image_name = "hls-foundation-inference"
image_tag = "latest"

# Build and push the Docker image
! az acr build --registry {acr_name} --image {image_name}:{image_tag} .

In [ ]:
# Create Azure ML Environment
environment_name = "hls-foundation-env"

environment = Environment(
    name=environment_name,
    description="Environment for HLS Foundation Model",
    image=f"{acr_name}.azurecr.io/{image_name}:{image_tag}",
)

ml_client.environments.create_or_update(environment)
print(f"Environment {environment_name} created/updated")


In [ ]:
# Register the model in Azure ML
model_name = "hls-foundation-model"

model = Model(
    name=model_name,
    description="HLS Foundation Model for satellite imagery analysis",
    path="./",  # Path to your model files
    type="custom_model"
)

registered_model = ml_client.models.create_or_update(model)
print(f"Model {model_name} registered with version {registered_model.version}")

In [ ]:
# Create managed online endpoint
endpoint_name = "hls-foundation-endpoint"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint for HLS Foundation Model inference",
    tags={"model": "hls-foundation", "type": "satellite-imagery"}
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"Endpoint {endpoint_name} created")

In [ ]:
# Set 100% traffic to the new deployment
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("Traffic set to 100% for the deployment")

In [ ]:
%%writefile test.json
{
  "instances": {
    "date": "2023-08-13",
    "bounding_box": [-156.81605703476012, 20.69675592885614, -156.41605703476014, 21.096755928856137],
    "model_id": "burn_scars"
  }
}

In [ ]:
# Test the deployed endpoint
import json

# Load test data
with open("test.json", "r") as f:
    test_data = json.load(f)

# Make prediction
try:
    response = ml_client.online_endpoints.invoke(
        endpoint_name=endpoint_name,
        deployment_name=deployment_name,
        request_file="test.json"
    )
    print("Prediction response:")
    print(response)
except Exception as e:
    print(f"Error during prediction: {e}")

In [ ]:
# Alternative: Test endpoint using REST API
import requests
import json
from azure.identity import DefaultAzureCredential

# Get access token
credential = DefaultAzureCredential()
token = credential.get_token("https://ml.azure.com/.default").token

# Endpoint URL
endpoint_url = f"https://{endpoint_name}.{ml_client.workspace_name}.inference.ml.azure.com/score"

# Headers
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Load and send test data
with open("test.json", "r") as f:
    test_data = json.load(f)

response = requests.post(endpoint_url, headers=headers, json=test_data)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

In [ ]:
# Get endpoint details and logs
endpoint_details = ml_client.online_endpoints.get(endpoint_name)
print(f"Endpoint Status: {endpoint_details.provisioning_state}")
print(f"Endpoint URI: {endpoint_details.scoring_uri}")

# Get deployment logs
logs = ml_client.online_deployments.get_logs(
    name=deployment_name,
    endpoint_name=endpoint_name,
    lines=50
)
print("Recent logs:")
print(logs)

In [ ]:
# Cleanup resources (uncomment to run)
# ml_client.online_endpoints.begin_delete(name=endpoint_name).result()
# print(f"Endpoint {endpoint_name} deleted")